In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics.pairwise import cosine_similarity
import os

In [32]:
train_directory = '/kaggle/input/sign-data-cut-v2/sign_data_cutv2/train'
test_directory = '/kaggle/input/sign-data-cut-v2/sign_data_cutv2/test'

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 0.1,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.1
)

train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size = (224, 224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size=32
    
)


test_datagen = ImageDataGenerator(
    rescale = 1./255,
)

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size = (224, 224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size=32
)

Found 639 images belonging to 54 classes.
Found 128 images belonging to 54 classes.


In [33]:
from tensorflow.keras.applications.vgg16 import VGG16

vgg_basemodel = VGG16(include_top=True)

In [34]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
#checkpoint = ModelCheckpoint('vgg16_finetune.h15', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)

In [35]:
vgg_model = tf.keras.Sequential(vgg_basemodel.layers[:-1])
vgg_model.add(tf.keras.layers.Dense(54, activation = 'softmax'))

# Freezing original layers
for layer in vgg_model.layers[:-1]:
    layer.trainable = False

vgg_model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.001, decay=0.01),
                  metrics=['accuracy'])

history = vgg_model.fit(train_generator,
              epochs=40,
              batch_size=64,
              validation_data=test_generator,
              callbacks=[early_stopping, lr_reduce])

# finetuning with all layers set trainable

for layer in vgg_model.layers:
    layer.trainable = True

vgg_model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.0001),
                  metrics=['accuracy'])

history2 = vgg_model.fit(train_generator,
              epochs=8,
              batch_size=64,
              validation_data=test_generator,
              callbacks=[early_stopping, lr_reduce])

vgg_model.save('/kaggle/working/saved_models/vgg_finetuned_model.keras')

Epoch 1/40
 2/20 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 0.0312 - loss: 4.7582

W0000 00:00:1715536108.444535     211 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


12/20 ━━━━━━━━━━━━━━━━━━━━ 3s 464ms/step - accuracy: 0.0243 - loss: 4.4200

W0000 00:00:1715536113.486997     211 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step - accuracy: 0.0226 - loss: 4.3788

...

17/20 ━━━━━━━━━━━━━━━━━━━━ 1s 501ms/step - accuracy: 0.7475 - loss: 1.5817

W0000 00:00:1715536558.916088     211 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step - accuracy: 0.7487 - loss: 1.5428

W0000 00:00:1715536560.916553     209 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 538ms/step - accuracy: 0.7492 - loss: 1.5318 - val_accuracy: 0.8750 - val_loss: 0.7230 - learning_rate: 1.0000e-04
Epoch 2/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 410ms/step - accuracy: 0.8772 - loss: 0.7310 - val_accuracy: 0.9453 - val_loss: 0.3837 - learning_rate: 1.0000e-04
Epoch 3/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 443ms/step - accuracy: 0.9323 - loss: 0.4183 - val_accuracy: 0.9688 - val_loss: 0.2154 - learning_rate: 1.0000e-04
Epoch 4/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 410ms/step - accuracy: 0.9744 - loss: 0.2369 - val_accuracy: 0.9922 - val_loss: 0.1060 - learning_rate: 1.0000e-04
Epoch 5/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 423ms/step - accuracy: 0.9717 - loss: 0.1516 - val_accuracy: 1.0000 - val_loss: 0.0692 - learning_rate: 1.0000e-04
Epoch 6/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 413ms/step - accuracy: 0.9835 - loss: 0.0969 - val_accuracy: 0.9609 - val_loss: 0.1172 - learning_rate: 1.0000e-04
Epoch 7/8
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 416ms/step - accuracy: 0.9905 - l

In [27]:
vgg_model = tf.keras.models.load_model('/kaggle/working/saved_models/vgg_finetuned_model.keras')
feature_extractor = tf.keras.Sequential(vgg_model.layers[:-1])

In [29]:
def load_image(image_path):
    '''Return the image in the format required by VGG16 model.'''
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    return x

def extract_features(paths):
    '''Returns the features extracted by the model.'''
    feature_list = []
    i = 0
    for images in paths:
        i += 1
        features = {}
        features['anchor'] = feature_extractor.predict(load_image(images[0]))
        features['negative'] = feature_extractor.predict(load_image(images[1]))
        features['positive'] = feature_extractor.predict(load_image(images[2]))
        feature_list.append(features)
        
    return feature_list

def cosine_similarity_fn(feature_list):
    '''Returns the cosine similarity between the two signature images.'''
    cosine_similarities = []
    for path in feature_list:
        similarities = {}
        similarities['matching_pair_score'] = cosine_similarity(path['positive'], path['anchor'])[0][0]
        similarities['non_matching_pair_score'] = cosine_similarity(path['negative'], path['anchor'])[0][0]
        cosine_similarities.append(similarities)
    return cosine_similarities


root = '/kaggle/input/sign-data-cut-v2/sign_data_cutv2/test'
image_set = []
directories = sorted(os.listdir(root))

for directory in directories:
    current_root = os.path.join(root, directory)
    image_set.append([os.path.join(current_root, img) for img in sorted(os.listdir(current_root))])

feature_set = extract_features(image_set)
cosine_similarities = cosine_similarity_fn(feature_set)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


IndexError: list index out of range